# Description
This notebook will experiment with:
- train & compare differnt GNN models for graph classificaiton in common benchmarks (PPI, Proteins, ENZYMES,..)
- compare results to publication results

Most of the experiments will be done in PyTorch/PyTorch Geometric, but some models are implemented in Tensor Flow.

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import MessagePassing
#from torch_geometric.nn.conv.gated_graph_conv import GatedGraphConv
from torch_geometric.nn.glob.glob import global_mean_pool, global_add_pool
import torch.nn as nn

from TFM_graph_classification import *

# 1. Models

In [2]:

class Net1(torch.nn.Module):
    def __init__(self, d1=50,d2=20,num_classes=6, num_layers=2, aggr_type='mean'):
        super(Net1, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=d1, num_layers=num_layers,aggr=aggr_type, bias=True)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, num_classes)
        self.global_pool = global_mean_pool
        
        

    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        #x = self.fc1(x)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        #x = self.pool1(x, batch )
        x = F.log_softmax(x, dim=1)
        #x = torch.argmax(x, dim=1)  # we output softmax to use the nll_loss
        
        return x
    
class Net2(torch.nn.Module):
    def __init__(self, d1=50,d2=20,num_classes=6, num_layers=2, aggr_type='mean'):
        super(Net2, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=d1, num_layers=num_layers,aggr=aggr_type, bias=True)
        self.fc1 = nn.Linear(d1, num_classes)
        self.global_pool = global_mean_pool
        
    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        
        x = self.fc1(x)
        x = F.log_softmax(x, dim=1)
        return x
    
    
class Net3(torch.nn.Module):
    def __init__(self, d1=50,d2=20, d3=10,num_classes=6, num_layers=2, aggr_type='mean'):
        super(Net3, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=d1, num_layers=num_layers,aggr=aggr_type, bias=True)
        self.fc1 = nn.Linear(d1, d2)
        self.dense1_bn = nn.BatchNorm1d(d2)
        self.fc2 = nn.Linear(d2, d3)
        self.dense2_bn = nn.BatchNorm1d(d3)
        self.fc3 = nn.Linear(d3, num_classes)
        self.global_pool = global_mean_pool
        
    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        x = F.relu(self.dense1_bn(self.fc1(x)))
        x = F.relu(self.dense2_bn(self.fc2(x)))
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x
    
class Net4(torch.nn.Module):
    def __init__(self, d1=50,d2=20,num_classes=6, num_layers=2, aggr_type='mean'):
        super(Net4, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=d1, num_layers=num_layers,aggr=aggr_type, bias=True)
        self.fc1 = nn.Linear(d1, d2)
        self.dense1_bn = nn.BatchNorm1d(d2)
        self.fc2 = nn.Linear(d2, num_classes)
        self.global_pool = global_mean_pool
        
        

    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        #x = self.fc1(x)
        x = F.relu(self.dense1_bn(self.fc1(x)))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        #x = self.pool1(x, batch )
        x = F.log_softmax(x, dim=1)
        #x = torch.argmax(x, dim=1)  # we output softmax to use the nll_loss
        
        return x

In [3]:
# models for hyperparameter search
model_list =[
    {'epochs': 200,
    'model': Net1,
    'kwargs':{'d1': 50,'d2': 20,'num_layers':2, 'aggr_type':'mean'}, 
    'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 32},
    {'epochs': 100,
    'model': Net1,
    'kwargs':{'d1': 50,'d2': 20,'num_layers':2, 'aggr_type':'mean'}, 
    'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 32},
    {'epochs': 200,
    'model': Net1,
    'kwargs':{'d1': 50,'d2': 20,'num_layers':2, 'aggr_type':'add'}, 
    'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 32},
    {'epochs': 100,
    'model': Net1,
    'kwargs':{'d1': 50,'d2': 20,'num_layers':2, 'aggr_type':'add'}, 
    'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 32},
    
    {'epochs': 200,
    'model': Net1,
    'kwargs':{'d1': 100,'d2': 20,'num_layers':2, 'aggr_type':'mean'}, 
    'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 32},
    {'epochs': 100,
    'model': Net1,
    'kwargs':{'d1': 100,'d2': 20,'num_layers':2, 'aggr_type':'mean'}, 
    'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 32},
    {'epochs': 200,
    'model': Net1,
    'kwargs':{'d1': 100,'d2': 50,'num_layers':2, 'aggr_type':'mean'}, 
    'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 32},
    {'epochs': 100,
    'model': Net1,
    'kwargs':{'d1': 100,'d2': 50,'num_layers':2, 'aggr_type':'mean'}, 
    'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 32},
]

model_list2 = []
for modelclass in [Net1, Net2, Net3,Net4]:
    for d1 in [25,50,100,200]:
        for d2 in [20,50]:
            for aggr_type in ['mean','add']:
                for epochs in [10,20,300]:
                    model_list2.append(
                        {
                        'model': modelclass,
                        'epochs': epochs,
                        'kwargs':{'d1': d1,'d2': d2,'num_layers':2, 
                                  'aggr_type':aggr_type}, 
                        'learning_rate': 0.01, 'weight_decay':5e-4, 
                        'batch_size': 32},
                    )
                    
model_list = model_list2
model_list = model_list[:2]

In [4]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
dataset = dataset.shuffle()
k = 3
n = len(dataset)
print(" n:",n," k folds=",k)
train_dataset, test_dataset = balancedDatasetSplit_slice(dataset, prop=0.8)
print("Datasets balancing: ")
printDatasetBalance(dataset )
printDatasetBalance(train_dataset )
printDatasetBalance(test_dataset )
print()

modelsdict = modelSelection(model_list,k, train_dataset)
reportModelSelectionResult(modelsdict)

 n: 600  k folds= 3
Datasets balancing: 
{0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100}
{0: 80, 1: 80, 2: 80, 3: 80, 4: 80, 5: 80}
{0: 20, 1: 20, 2: 20, 3: 20, 4: 20, 5: 20}

 trained model:  type {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 10  val loss= 0.05398473019401232  val accuracy= 0.35714285714285715  val microF1= 0.28846153846153844  val macroF1= 0.24986523196358376
 trained model:  type {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 20  val loss= 0.0538820947209994  val accuracy= 0.25  val microF1= 0.2948717948717949  val macroF1= 0.27415437042061647

 selected model from loss:  Net1 {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 20 0.0538820947209994 0.25 0.2948717948717949 0.27415437042061647
 selected model from accuracy:  Net1 {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 10 0.05398473019401232 0.35714285714285715 0.2948717948717949 0.27415437042061647
 selected model from microF1:  

In [5]:
bmodel = final_model_train(modelsdict['best_models']['loss'], train_dataset)
testresult = testModel(bmodel, test_dataset)
modelsdict['testing'][bmodel.__class__.__name__+'loss']=testresult

len(test_dataset):  120
num graphs:  120
tensor([5, 5, 1, 5, 2, 5, 5, 1, 5, 5, 2, 5, 5, 5, 5, 1, 1, 2, 5, 2, 1, 0, 2, 5,
        5, 4, 2, 2, 2, 2, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 2, 4, 0, 5, 5, 0, 2, 0,
        0, 5, 2, 4, 5, 4, 2, 5, 5, 0, 4, 5, 2, 4, 4, 5, 5, 4, 1, 5, 0, 2, 2, 4,
        5, 1, 5, 1, 5, 0, 2, 5, 2, 5, 4, 4, 5, 1, 1, 5, 1, 0, 5, 5, 4, 2, 4, 5,
        5, 4, 4, 4, 5, 5, 4, 4, 4, 4, 2, 5, 5, 4, 5, 2, 5, 5, 4, 5, 5, 1, 5, 5],
       device='cuda:0')
tensor([3, 0, 1, 4, 4, 4, 3, 1, 3, 3, 4, 4, 2, 1, 5, 1, 1, 2, 3, 2, 5, 0, 5, 3,
        0, 0, 2, 2, 2, 2, 1, 2, 1, 4, 5, 5, 1, 1, 2, 5, 2, 4, 0, 0, 2, 5, 2, 1,
        3, 0, 2, 0, 5, 4, 0, 2, 5, 5, 4, 4, 1, 0, 3, 0, 4, 4, 5, 5, 2, 3, 2, 4,
        0, 5, 0, 3, 5, 1, 0, 1, 2, 4, 3, 1, 4, 3, 4, 5, 0, 1, 5, 1, 3, 2, 3, 1,
        3, 2, 0, 4, 5, 0, 3, 5, 1, 3, 4, 1, 0, 0, 5, 3, 4, 3, 0, 3, 1, 2, 5, 4],
       device='cuda:0')
Accuracy: 0.3167  macroF1: 0.29489583333333336  microF1: 0.31666666666666665


In [6]:
bmodel = final_model_train(modelsdict['best_models']['accuracy'], train_dataset)
testresult = testModel(bmodel, test_dataset)
modelsdict['testing'][bmodel.__class__.__name__+'accuracy']=testresult

len(test_dataset):  120
num graphs:  120
tensor([5, 5, 5, 5, 5, 2, 5, 5, 5, 2, 5, 3, 2, 5, 2, 4, 5, 5, 5, 5, 2, 5, 5, 2,
        2, 2, 5, 2, 2, 5, 5, 2, 5, 2, 5, 5, 2, 5, 5, 5, 5, 2, 2, 5, 2, 2, 5, 5,
        2, 2, 2, 3, 5, 5, 2, 2, 5, 3, 5, 5, 5, 5, 2, 2, 1, 5, 5, 3, 4, 5, 1, 2,
        2, 3, 2, 2, 5, 5, 3, 5, 2, 2, 2, 5, 3, 2, 2, 2, 5, 5, 5, 1, 2, 2, 2, 5,
        5, 5, 5, 2, 2, 3, 1, 1, 2, 5, 5, 5, 5, 5, 5, 2, 2, 3, 5, 2, 2, 2, 4, 5],
       device='cuda:0')
tensor([4, 4, 1, 1, 3, 1, 4, 1, 0, 1, 1, 1, 5, 5, 0, 4, 3, 3, 5, 5, 0, 3, 5, 4,
        3, 5, 1, 4, 2, 5, 4, 0, 0, 2, 0, 4, 0, 2, 1, 3, 0, 3, 3, 0, 4, 4, 3, 1,
        0, 2, 2, 5, 3, 5, 2, 3, 5, 5, 1, 4, 2, 0, 2, 2, 3, 3, 5, 2, 1, 1, 3, 0,
        0, 4, 2, 3, 5, 2, 1, 4, 2, 0, 2, 0, 5, 1, 3, 4, 5, 2, 3, 1, 3, 0, 2, 4,
        2, 5, 4, 3, 2, 5, 1, 3, 4, 0, 5, 1, 0, 5, 0, 1, 4, 1, 5, 4, 2, 2, 0, 4],
       device='cuda:0')
Accuracy: 0.2583  macroF1: 0.23426782859330317  microF1: 0.25833333333333336


In [7]:
bmodel = final_model_train(modelsdict['best_models']['microF1'], train_dataset)
testresult = testModel(bmodel, test_dataset)
modelsdict['testing'][bmodel.__class__.__name__+'microF1']=testresult

len(test_dataset):  120
num graphs:  120
tensor([2, 1, 1, 4, 5, 1, 0, 2, 4, 0, 5, 1, 5, 2, 1, 2, 0, 0, 2, 2, 5, 5, 2, 0,
        2, 2, 2, 5, 1, 4, 0, 4, 1, 4, 2, 4, 4, 0, 2, 5, 2, 5, 5, 5, 1, 2, 2, 2,
        1, 5, 2, 1, 4, 1, 2, 0, 2, 5, 1, 5, 2, 1, 2, 2, 1, 5, 2, 5, 5, 4, 1, 2,
        4, 4, 1, 1, 2, 2, 0, 2, 0, 4, 2, 1, 2, 3, 4, 2, 1, 5, 1, 2, 3, 1, 2, 2,
        0, 1, 5, 1, 4, 1, 0, 5, 1, 1, 1, 0, 4, 2, 2, 2, 2, 5, 5, 4, 4, 5, 5, 2],
       device='cuda:0')
tensor([2, 3, 2, 0, 4, 4, 5, 5, 3, 5, 5, 5, 5, 0, 4, 2, 4, 0, 3, 2, 2, 3, 0, 1,
        1, 2, 2, 5, 5, 4, 5, 5, 3, 4, 2, 0, 4, 1, 4, 0, 4, 4, 4, 5, 5, 1, 0, 2,
        4, 3, 2, 1, 1, 1, 3, 0, 4, 5, 0, 2, 4, 1, 5, 2, 4, 5, 3, 5, 4, 3, 1, 1,
        0, 3, 3, 1, 2, 1, 5, 2, 2, 0, 3, 3, 1, 3, 0, 0, 1, 0, 3, 5, 3, 0, 2, 4,
        1, 3, 2, 3, 1, 1, 0, 2, 5, 1, 0, 1, 0, 0, 2, 3, 1, 4, 5, 4, 0, 3, 4, 2],
       device='cuda:0')
Accuracy: 0.3250  macroF1: 0.36379260753217935  microF1: 0.325


In [8]:
bmodel = final_model_train(modelsdict['best_models']['macroF1'], train_dataset)
testresult = testModel(bmodel, test_dataset)
modelsdict['testing'][bmodel.__class__.__name__+'macroF1']=testresult

len(test_dataset):  120
num graphs:  120
tensor([4, 1, 2, 3, 1, 1, 1, 1, 2, 2, 2, 2, 1, 3, 1, 2, 1, 4, 0, 2, 1, 1, 2, 2,
        1, 2, 4, 1, 2, 0, 3, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 1, 2, 2, 2, 3,
        2, 1, 2, 0, 2, 1, 3, 1, 1, 4, 1, 4, 2, 3, 4, 2, 2, 2, 2, 2, 4, 2, 0, 1,
        4, 5, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 2, 3, 1, 2, 2, 2, 2, 2, 5, 1, 0, 1,
        0, 2, 1, 1, 1, 3, 0, 3, 1, 1, 1, 3, 2, 0, 2, 1, 2, 2, 1, 4, 1, 1, 1, 2],
       device='cuda:0')
tensor([4, 4, 1, 3, 3, 1, 1, 3, 2, 2, 1, 0, 5, 0, 4, 2, 0, 1, 0, 3, 3, 1, 1, 1,
        3, 3, 1, 0, 2, 2, 0, 4, 5, 5, 1, 5, 2, 1, 0, 2, 5, 1, 3, 2, 2, 0, 3, 3,
        3, 3, 4, 0, 0, 5, 0, 2, 4, 1, 3, 2, 0, 0, 3, 2, 2, 4, 3, 2, 0, 2, 4, 5,
        5, 4, 1, 5, 1, 3, 4, 0, 3, 0, 4, 1, 4, 4, 0, 4, 2, 5, 2, 5, 5, 0, 5, 3,
        5, 2, 5, 2, 4, 0, 1, 4, 3, 4, 5, 4, 3, 5, 0, 1, 5, 2, 4, 5, 1, 5, 1, 4],
       device='cuda:0')
Accuracy: 0.2583  macroF1: 0.266778352575742  microF1: 0.25833333333333336


In [9]:
reportAllTest(modelsdict)
saveResults(modelsdict)
# review microF1 & macroF1
# review Random Baseline

,accuracy,macroF1,microF1,name
0,0.316667,0.294896,0.316667,Net1loss
1,0.258333,0.234268,0.258333,Net1accuracy
2,0.325000,0.363793,0.325000,Net1microF1
3,0.258333,0.266778,0.258333,Net1macroF1


### Results
1. encapsulate all training, model selection,.. everything
2. present results with Pandas tables, and histograms
3. save models and results to disk, and load them later for testing
4. transform into a python module or package

### Pending:

- prepare another notebook using the python module (prepare local and on collab)
- test other GNN layers: GAT, GCN, GraphSAGE, Metalayer
- do a good HP search
- look for published architectures?
- compare with published benchmarks
